#  Probabilistic Matrix Factorization of the MovieLens Ratings

Something something . Matrix factorization is a simple and powerful technique for predicting user's ratings of items using embedding in a latent space.

Modern methods have many useful adaptations, but the general system predicts user $u$'s rating of item $i$, $r_{u,i}$  as 
  *  $r_{u,i} = \mu + \beta_u + \beta_i + \vec{v}_u^T \cdot \vec{v}_i^T
  *  $\mu$, $\beta_u$, and $\beta_i$ are overall mean ratings and offsets for users and movies
  *  $\vec{v}_i, \vec{v}_u \in \mathbb{R}^K$ are the user's and movie's embeddings in a $K$ dimensional space.
  *  We regularize user and item vectors by modeling them as coming from a multivariate gaussian, which can be learned or predetermined $\vec{v}_i \sim N(0, \Lambda_{\text{Movies}})$ and $\vec{v}_i \sim N(0, \Lambda_{\text{Movies}})$.

The most famous application of PMF for recomendation systems is probably Netflix's.  Their system is described in [this paper](https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf) and you can also find a readable overview [here](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf).  Netflix took down the original dataset due to privacy concerns, but you can still run the same model using the [MovieLens data](https://grouplens.org/datasets/movielens/20m/).

Need to cite `MovieLens` with ```F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872```.

The MovieLens data comes contains 20,000,263 ratings of 27,278 movies by 138,493 users. It also contains free text tags, but we will not use them here. 

##  Read in the Data

Download [the data](https://grouplens.org/datasets/movielens/) and unzip.

We will only use `ml-20/movies.csv` and `ml-20/ratings.csv`.

In [29]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

%pylab inline

movies = pd.read_csv('ml-20m/movies.csv')
ratings = pd.read_csv('ml-20m/ratings.csv')

In [187]:
print(movies.shape)
print(movies.head())

(27278, 3)
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [188]:
print(ratings.shape)
print(ratings.head())

(20000263, 4)
   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580


Let's cherry pick a few movies whose parameters we will track.  We'll pick the ones with the most ratings - hopefully our latent dimensions will be interpretable. 

We'll also bring in the titles now.  We'll need to remap the user and movie IDs, and bringing in the titles helps make sure we don't make mistakes mapping them back later on.

In [190]:
ratings = ratings.merge(movies, on = 'movieId', how = 'left')

In [192]:
print(ratings.head())

   userId  movieId  rating   timestamp  \
0       1        2     3.5  1112486027   
1       1       29     3.5  1112484676   
2       1       32     3.5  1112484819   
3       1       47     3.5  1112484727   
4       1       50     3.5  1112484580   

                                               title  \
0                                     Jumanji (1995)   
1  City of Lost Children, The (Cité des enfants p...   
2          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
3                        Seven (a.k.a. Se7en) (1995)   
4                         Usual Suspects, The (1995)   

                                   genres  
0              Adventure|Children|Fantasy  
1  Adventure|Drama|Fantasy|Mystery|Sci-Fi  
2                 Mystery|Sci-Fi|Thriller  
3                        Mystery|Thriller  
4                  Crime|Mystery|Thriller  


In [194]:
ratings['title'].value_counts()[0:5]

Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: title, dtype: int64

In [197]:
ratings.loc[ratings['rating'] >= 4.0]['title'].value_counts()[0:5]

Shawshank Redemption, The (1994)             55807
Pulp Fiction (1994)                          52353
Silence of the Lambs, The (1991)             50114
Forrest Gump (1994)                          47331
Star Wars: Episode IV - A New Hope (1977)    42612
Name: title, dtype: int64

In [198]:
ratings.loc[ratings['rating'] <= 1.0]['title'].value_counts()[0:5]

Dumb & Dumber (Dumb and Dumber) (1994)    4578
Ace Ventura: Pet Detective (1994)         4323
Ace Ventura: When Nature Calls (1995)     3976
Waterworld (1995)                         3013
Blair Witch Project, The (1999)           2992
Name: title, dtype: int64

In [200]:
_, _, _ = plt.hist(np.log(ratings['title'].value_counts()), 30)

In [206]:
print(ratings['rating'].value_counts())

4.0    5561926
3.0    4291193
5.0    2898660
3.5    2200156
4.5    1534824
2.0    1430997
2.5     883398
1.0     680732
1.5     279252
0.5     239125
Name: rating, dtype: int64


In [ ]:
easier_ratings = ratings.loc[ratings

##  Model Definition

Let's go ahead and define our model before we reformat our data.

We will use a vanilla PMF - exactly the one defined above, with predetermined $\Lambda_{\text{movies}} = \Lambda_{\text{users}} = I_K$ and $K=3$.

In [217]:
from edward.models import Normal
#  This will need to change if you train/test split
N_users = len(set(ratings.userId))
N_movies = len(set(ratings.movieId))
N_ratings = ratings.shape[0]
K = 2

lnvar_users = Normal(loc=tf.zeros([1]), scale=tf.ones([1]))
lnvar_movies = Normal(loc=tf.zeros([1]), scale=tf.ones([1]))
sigma_users = tf.sqrt(tf.exp(lnvar_users))
sigma_movies = tf.sqrt(tf.exp(lnvar_users))

user_vecs = Normal(loc = tf.zeros([N_users, K]), 
                   scale = sigma_users * tf.ones([N_users, K]))
movie_vecs = Normal(loc = tf.zeros([N_movies, K]), 
                    scale = sigma_movies * tf.ones([N_movies, K]))

#  Somewhat hacky prior on mu
mu = Normal(loc = 2.5*tf.ones([1]), 
            scale = tf.ones([1]))

user_betas = Normal(loc = tf.zeros([N_users]), 
                    scale = sigma_users * tf.ones([N_users]))
movie_betas = Normal(loc = tf.zeros([N_movies]), 
                     scale = sigma_movies * tf.ones([N_movies]))

#  Placeholders for data inputs
user_ids = tf.placeholder(tf.int32, [N_ratings])
movie_ids = tf.placeholder(tf.int32, [N_ratings])

predicted_ratings = tf.reduce_sum(tf.multiply(
    tf.gather(user_vecs, user_ids),
    tf.gather(movie_vecs, movie_ids)
)) + \
    tf.gather(user_betas, user_ids) + \
    tf.gather(movie_betas, movie_ids) + \
    mu

obs_ratings = Normal(loc=predicted_ratings, scale = tf.ones([N_ratings]))

##  Inference Definition

We have now sret up a probablistic graph for generating ratings from our learnable paramters (mu, offsets, and vectors).  We now explicitly define our inference.  Edward makes it easy to swap out sampling, ML, and variational methods.  We'll use simple MFVI.  

In [218]:
user_ids_train = ratings['userId']
movie_ids_train = ratings['movieId'].astype('category').cat.codes

user_ids_train = user_ids_train.values.astype(int) - 1
movie_ids_train = movie_ids_train.values.astype(int)
ratings_train = ratings['rating'].values.astype(float)

In [219]:
# INFERENCE
q_user_vecs = Normal(loc=tf.Variable(tf.random_normal([N_users, K])),
                     scale=tf.nn.softplus(tf.Variable(tf.random_normal([N_users, K]))))
q_movie_vecs = Normal(loc=tf.Variable(tf.random_normal([N_movies, K])),
                      scale=tf.nn.softplus(tf.Variable(tf.random_normal([N_movies, K]))))
q_user_betas = Normal(loc=tf.Variable(tf.random_normal([N_users])),
                      scale=tf.nn.softplus(tf.Variable(tf.random_normal([N_users]))))
q_movie_betas = Normal(loc=tf.Variable(tf.random_normal([N_movies])),
                      scale=tf.nn.softplus(tf.Variable(tf.random_normal([N_movies]))))

q_mu = Normal(loc=tf.Variable(tf.random_normal([1])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))
q_lnvar_users = Normal(loc=tf.Variable(tf.random_normal([1])),
                       scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))
q_lnvar_movies = Normal(loc=tf.Variable(tf.random_normal([1])),
                        scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

                                    
parameter_inferences = {
    user_vecs: q_user_vecs,
    movie_vecs: q_movie_vecs,
    user_betas: q_user_betas,
    movie_betas: q_movie_betas,
    mu: q_mu,
    lnvar_users: q_lnvar_users,
    lnvar_movies: q_lnvar_movies
}
train_data = {
    user_ids: user_ids_train,
    movie_ids: movie_ids_train,
    obs_ratings: ratings_train
}

inference = ed.KLqp(parameter_inferences,
                    train_data)


This needs to go further up.

Now let's plug in our data into the data placeholders.

We also want to check that the IDs make don't skip any indices, which can cause problems.  We need our actual IDs to match up with the shapes of our parameters.

In [ ]:
print(max(ratings['userId']))
print(len(set(ratings['userId'])))
print(max(ratings['movieId']))
print(len(set(ratings['movieId'])))


In [73]:
print(min(set(ratings['movieId'])))
print(min(set(ratings['userId'])))

1
1


#  Run the Inference

We now have everything set up and can let SGD run. 

In [220]:
optimizer = tf.train.AdamOptimizer()

inference.initialize(optimizer = optimizer,
                     n_print=5, n_iter=100)
tf.global_variables_initializer().run()

In [221]:
for _ in range(inference.n_iter):
  info_dict = inference.update()
  inference.print_progress(info_dict)

500/500 [100%] ██████████████████████████████ Elapsed: 608s | Loss: 91254071230464.000

495/500 [ 99%] █████████████████████████████  ETA: 6s | Loss: 1589547800461312.000

490/500 [ 98%] █████████████████████████████  ETA: 12s | Loss: 49143032578048.000 

485/500 [ 97%] █████████████████████████████  ETA: 18s | Loss: 628197150425088.000

480/500 [ 96%] ████████████████████████████   ETA: 24s | Loss: 716284043984896.000

...

 20/500 [  4%] █                              ETA: 1022s | Loss: 2793779494912.000

 15/500 [  3%]                                ETA: 1179s | Loss: 314555016151040.000

 10/500 [  2%]                                ETA: 1478s | Loss: 262545176788992.000

  5/500 [  1%]                                ETA: 2384s | Loss: 2037256290304.000  

  1/500 [  0%]                                ETA: 9607s | Loss: 418365516546048.000

#  Model Criticism

Just as a first sanity check, let's see which movies the model thinks are best. 

In [222]:
fit_movie_means = q_movie_betas.mean().eval()

In [223]:
movie_categories_df['fit_beta'] = fit_movie_means

In [224]:
movie_betas_df = movie_categories_df.merge(movies, on = 'movieId', how='left')

In [225]:
movie_betas_df.sort_values(['fit_beta']).head()

       movieId  movie_cat_code  fit_beta                                title  \
14952    75397           14952 -3.919719                   Torrid Zone (1940)   
17525    88349           17525 -3.876275                 Almighty Thor (2011)   
14079    70740           14079 -3.676230          I Really Hate My Job (2007)   
1452      1495            1452 -3.641302  Turbo: A Power Rangers Movie (1997)   
6947      7059            6947 -3.638973               National Velvet (1944)   

                          genres  
14952    Action|Adventure|Comedy  
17525          Adventure|Fantasy  
14079               Comedy|Drama  
1452   Action|Adventure|Children  
6947              Children|Drama  

In [226]:
movie_betas_df.sort_values(['fit_beta'], ascending=False).head(5)

       movieId  movie_cat_code  fit_beta                            title  \
26294   128908           26294  4.028948                Cloudburst (2011)   
17467    88127           17467  3.924351  Conan O'Brien Can't Stop (2011)   
22494   108255           22494  3.912767  Venus Wars (Venus Senki) (1989)   
24846   118776           24846  3.887387                 Lap Dance (2014)   
10084    33330           10084  3.774137         Edges of the Lord (2001)   

                            genres  
26294       Adventure|Comedy|Drama  
17467                  Documentary  
22494  Action|Animation|Sci-Fi|War  
24846                        Drama  
10084                    Drama|War  

It pretty clearly looks to me like the model is not yet finding ratings that make any sense at all.  Let's see if it's putting anything into the latent dimensions.  None of the 'best' movies fit. 

In [ ]:
Let's also check the other learned parameters to see if they make sense.

In [234]:
print('Mu is at {}'.format(q_mu.mean().eval()))

print('Sigma for movies is at {}'.format(np.sqrt(np.exp(q_lnvar_users.mean().eval()))))
print('Sigma for users is at {}'.format(np.sqrt(np.exp(q_lnvar_movies.mean().eval()))))

Sigma for users is at [ 1.14207709]


Sigma for movies is at [ 1.50328493]


Mu is at [-0.06361528]


We can also check out the distribution of fit offsets for movies. Nothing too concerning.

In [133]:
_, _, _ = plt.hist(fit_movie_means, 30)

OK, so it does not look like I am grabbing the best movies.  

These are pretty much random, so I must be doing the IDs incorrectly.  I could also try displaying the most reviewed movies in two dimensions.  But let's first make sure I have the IDs right. 

#  Visualizing the Latent Dimensions

Now let's see if the factorization has learned meaningfull embeddings.

First let's pick a small subset of the movies to plot.

In [ ]:
movie_counts = ratings['movieId'].value_counts()